In [9]:
import dask, zarr, xarray, h5netcdf, h5pyd
import s3fs

from dask.diagnostics import ProgressBar

# TDS (single thread)

In [10]:
dask.config.set(scheduler='synchronous')

In [11]:
url = 'http://193.146.75.233:8080/thredds/dodsC/chunked/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_185001010030-185412312330.nc'
ds = xarray.open_dataset(url, engine='netcdf4', chunks={'time': 2739, 'lat': 8, 'lon': 32})

sync; echo 1 > /proc/sys/vm/drop_caches

In [12]:
with ProgressBar():
    ds['tas'].mean(dim='time').load()

[########################################] | 100% Completed |  5min 30.6s


## TDS (threads)

In [13]:
dask.config.set(scheduler='threads')

In [14]:
url = 'http://193.146.75.233:8080/thredds/dodsC/chunked/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_185001010030-185412312330.nc'
ds = xarray.open_dataset(url, engine='netcdf4', chunks={'time': 2739, 'lat': 8, 'lon': 32})

sync; echo 1 > /proc/sys/vm/drop_caches

In [15]:
with ProgressBar():
    ds['tas'].mean(dim='time').load()

[########################################] | 100% Completed |  4min 47.9s


# Zarr (single thread)

In [21]:
dask.config.set(scheduler='synchronous')

In [22]:
fs = s3fs.S3FileSystem(
  client_kwargs=dict(endpoint_url='https://cephrgw01.ifca.es:8080'),
  anon=False,
  key='9a1b9204bdc44acd9ca086b5026a5d18',
  secret='88167cd8f1034fecb2f37363f35fd505')

store = s3fs.S3Map(root='zarr/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_185001010030-185412312330', s3=fs, check=False)
ds = xarray.open_zarr(store)

sync; echo 1 > /proc/sys/vm/drop_caches

In [23]:
with ProgressBar():
    ds['tas'].mean(dim='time').load()

[########################################] | 100% Completed |  7min 55.4s


# Zarr (threads)

In [18]:
dask.config.set(scheduler='threads')

In [19]:
fs = s3fs.S3FileSystem(
  client_kwargs=dict(endpoint_url='https://cephrgw01.ifca.es:8080'),
  anon=False,
  key='9a1b9204bdc44acd9ca086b5026a5d18',
  secret='88167cd8f1034fecb2f37363f35fd505')

store = s3fs.S3Map(root='zarr/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_185001010030-185412312330', s3=fs, check=False)
ds = xarray.open_zarr(store)

sync; echo 1 > /proc/sys/vm/drop_caches

In [20]:
with ProgressBar():
    ds['tas'].mean(dim='time').load()

[########################################] | 100% Completed |  1min  0.0s


# HSDS

For this to work I had to:

- edit container's /etc/hosts and add hsds.hdf.test
- add the environment variables: HS_ENDPOINT="hsds.hdf.test" HS_USER HS_PASSWORD

In [7]:
dask.config.set(scheduler='threads')

In [8]:
url = 'http://hsds.hdf.test/home/public/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_185001010030-185412312330'
ds = xarray.open_dataset(url, engine='h5netcdf', decode_cf=False)

sync; echo 1 > /proc/sys/vm/drop_caches

In [15]:
# HSDS = Highly Scalable Data Service...
with ProgressBar():
    ds['tas'].mean(dim='time').load()

OSError: Error retrieving data: 413

In [14]:
ds

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 128, lon: 256, time: 43824)
Coordinates:
  * lat          (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon          (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
  * time         (time) float64 0.02083 0.0625 0.1042 ... 1.826e+03 1.826e+03
Dimensions without coordinates: axis_nbounds
Data variables:
    height       float64 ...
    tas          (time, lat, lon) float32 ...
    time_bounds  (time, axis_nbounds) float64 ...
Attributes:
    CMIP6_CV_version:       cv=6.2.3.0-7-g2019642
    Conventions:            CF-1.7 CMIP-6.2
    EXPID:                  CNRM-ESM2-1_historical_r1i1p1f2_v2
    activity_id:            CMIP
    arpege_minor_version:   6.3.2
    branch_method:          standard
    branch_time_in_child:   [0.]
    branch_time_in_parent:  [0.]
    contact:                contact.cmip@meteo.fr
    creation_date:          2018-09-15T06:24:21Z
    data_specs_version:     01.00.21
 